##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Convolutional Neural Network (CNN)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/cnn">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ar/tutorials/images/cnn.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />تفاعل مع المحتوى على Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ar/tutorials/images/cnn.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />اعرض المصدر على Github</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ar/tutorials/images/cnn.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

في هذا الدرس سيتم شرح تدريب بسيط لشبكة عصبية [Convolutional Neural Network](https://developers.google.com/machine-learning/glossary/#convolutional_neural_network) لتصنيف [CIFAR images](https://www.cs.toronto.edu/~kriz/cifar.html).  [Keras Sequential API](https://www.tensorflow.org/guide/keras/overview), هذا الدرس يستخدم Keras Sequential API  لنتمكن من تدريب الشبكة العصبية في بضع سطور .


### نبدأ بتضمين TensorFlow

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### نقوم بتحميل مجموعة الصور CIFAR 10

مجموعة الصور هذه تتكون من 60 ألف صورة ملونة منقسمين إلي 10 أقسام كل قسم منهم يحتوي علي 6 ألاف صورة. هذه المجموعة تتكون من 50 ألف صورة للتدريب و 10 ألاف صورة لإختبار الشبكة العصبية. لا يوجد تشابه بين الصور في التدريب والاختبار


In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### نتحقق من الصور

لنتحقق من أن الصور تبدو صحيحة. نبدأ بإظهار أول  25 صورة من مجموعة التدريب ونظهر اسم القسم التي تنتمي له كل صورة


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### نبدأ بإنشاء CNN

في أول 6 سطور من الكود نبدأ بتعريف بنمط متعارف عليه ألا وهو أكوام من الconvolutions و Max pooling  
 [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)  [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) .

المدخل للشبكة العصبية هو مصفوفة تتكون من (طول الصوره,عرض الصورة,عدد قنوات الألوان)
و قنوات الألوان تشير  إلي الأحمر و الأزرق و الأخضر .
في هذا المثال سنجعل المدخل يكون مصفوفة في شكل )32,32,3( لأن الصور ملونة و طولها و عرضها = 32 و يمكنك فعل ذك بتمرير  `input_shape` إلي أول طبقة

In [5]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

سنعرض طبقات و شكل الشبكة العصبية

In [ ]:
model.summary()

فوق, يمكنك أن تري أن المخرج من هذه الطبقات هو مصفوفة ثلاثية الأبعاد وتتكون من طول وعرض وعدد قنوات. الطول والعرض في نقصان كلما زادت الطبقات. أما عدد القنوات يمكنك التحكم فيه من أول معطي (مثال 32 أو 64) و يمكنك زيادة عدد القنوات لأن الطول والعرض للصورة في نقصان

### نبدأ في إضافة Dense layers 
لنكمل النموذج سنبدأ بإدخال المصفوفة الخارجة من طبقات الconvultions التي تخرج في شكل
(4,4,64) إلي dense layers لنقوم بالتصنيف بين الأقسام 
هذا الطبقات يكون المدخل لها هو متجه وهو أحادي البعد فيجب أن نقوم أولا بتحويل المصفوفة التي كانت ثلاثية الأبعاد إلي متجه و نضيف من هذه الطبقات و نجعل أخر طبقة تتكون من 10 وحدات وهو عدد الأقسام التي سيتم تصنيفهم 

In [7]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

هنا نعرض الشكل الكامل للشبكة العصبية

In [ ]:
model.summary()

نلاحظ أنه تم تحويل المصفوفة ثلاثية الأبعاد بالشكل (4,4,64) إلي متجه واحد يتكون من 1024 عنصر (4*4*64) 
و تم تحويله قبل أن يدخل علي الdense layers

### نقوم بتدريب النموذج

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

### نقوم بتقييم النموذج

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)

شبكتنا العصبية البسيطة حصلت علي دقة حوالي 70% و هو شيء ليس سيئا لبضع سطور من الكود 
يمكنك أن تري مثال أخر لكن باستخدام Keras subclassing APIو
 `tf.GradientTape` [هنا](https://www.tensorflow.org/tutorials/quickstart/advanced).